<a href="https://colab.research.google.com/github/kanakala2116/codealpha_tasks/blob/main/essay_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
df = pd.read_csv("/content/drive/MyDrive/training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1');
df.dropna(axis=1,inplace=True)
df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
df.head()
temp = pd.read_csv("/content/drive/MyDrive/Processed_data.csv")
temp.drop("Unnamed: 0",inplace=True,axis=1)

In [4]:
df['domain1_score']=temp['final_score']
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",6
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",5
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",8
4,5,1,"Dear @LOCATION1, I know having computers has a...",6


In [5]:
df['essay'][0]

"Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the econom

In [6]:
temp.head(1)

,essay_id,essay_set,essay,final_score,clean_essay,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
0,1,1,"Dear local newspaper, I think effects computer...",6,Dear local newspaper I think effects computer...,1441,344,16,4.188953,11,76,75,18,24


In [7]:
y = df['domain1_score']
df.drop('domain1_score',inplace=True,axis=1)
X=df

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
X_train.shape

(9083, 3)

In [10]:
train_e = X_train['essay'].tolist()
test_e = X_test['essay'].tolist()

In [11]:
train_sents=[]
test_sents=[]

stop_words = set(stopwords.words('english'))
def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = []
    words=x.split()
    for w in words:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

for i in train_e:
    train_sents+=essay2word(i)

for i in test_e:
    test_sents+=essay2word(i)

In [12]:
len(train_sents)

116500

In [13]:
train_sents[0]

['It',
 'first',
 'day',
 'high',
 'school',
 'gut',
 'full',
 'butterflies',
 'make',
 'want',
 'run',
 'bathrooms',
 'hide',
 'world']

In [14]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

In [15]:
from gensim.models import Word2Vec

# Define parameters
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

# Train Word2Vec model
model = Word2Vec(train_sents,
                 workers=num_workers,
                 vector_size=num_features,
                 min_count=min_word_count,
                 window=context,
                 sample=downsampling)

# Initialize vectors
model.init_sims(replace=True)

# Save the model
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)


<ipython-input-15-a6dd97f59e9d>:19: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [16]:
import numpy as np

def makeVec(words, model, num_features):
    vec = np.zeros((num_features,), dtype="float32")
    noOfWords = 0.
    index2word_set = set(model.wv.index_to_key)
    for word in words:
        if word in index2word_set:
            noOfWords += 1
            vec = np.add(vec, model.wv[word])
    if noOfWords > 0:
        vec = np.divide(vec, noOfWords)
    return vec

def getVecs(essays, model, num_features):
    essay_vecs = np.zeros((len(essays), num_features), dtype="float32")
    for i, essay in enumerate(essays):
        essay_vecs[i] = makeVec(essay, model, num_features)
    return essay_vecs

# Assuming sent2word is a function that processes sentences into words
clean_train = [sent2word(i) for i in train_e]
training_vectors = getVecs(clean_train, model, num_features)

clean_test = [sent2word(i) for i in test_e]
testing_vectors = getVecs(clean_test, model, num_features)


In [17]:
training_vectors.shape

(9083, 300)

In [18]:
training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 300)            721200    
                                                                 
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 814705 (3.11 MB)
Trainable params: 814705 (3.11 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
training_vectors.shape

(9083, 1, 300)

In [20]:
lstm_model.fit(training_vectors, y_train, batch_size=64, epochs=15)

Epoch 1/15
142/142 [==============================] - 11s 28ms/step - loss: 11.4979 - mae: 2.6609
Epoch 2/15
142/142 [==============================] - 6s 41ms/step - loss: 5.5052 - mae: 1.8568
Epoch 3/15
142/142 [==============================] - 4s 26ms/step - loss: 5.3436 - mae: 1.8330
Epoch 4/15
142/142 [==============================] - 4s 26ms/step - loss: 5.1701 - mae: 1.7927
Epoch 5/15
142/142 [==============================] - 5s 33ms/step - loss: 5.0854 - mae: 1.7767
Epoch 6/15
142/142 [==============================] - 5s 35ms/step - loss: 5.0801 - mae: 1.7835
Epoch 7/15
142/142 [==============================] - 4s 25ms/step - loss: 5.0371 - mae: 1.7716
Epoch 8/15
142/142 [==============================] - 4s 25ms/step - loss: 5.0704 - mae: 1.7812
Epoch 9/15
142/142 [==============================] - 5s 36ms/step - loss: 5.0151 - mae: 1.7649
Epoch 10/15
142/142 [==============================] - 5s 32ms/step - loss: 4.9604 - mae: 1.7531
Epoch 11/15
142/142 [================

In [21]:
lstm_model.save('final_lstm.h5')
y_pred = lstm_model.predict(testing_vectors)
y_pred = np.around(y_pred)
y_pred

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


122/122 [==============================] - 3s 12ms/step


array([[4.],
       [5.],
       [6.],
       ...,
       [7.],
       [6.],
       [8.]], dtype=float32)